<a href="https://colab.research.google.com/github/drivemax26/my_progect/blob/main/%D0%9A%D0%BE%D0%BF%D0%B8%D1%8F_%D0%B1%D0%BB%D0%BE%D0%BA%D0%BD%D0%BE%D1%82%D0%B0_%22Untitled0_ipynb%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import requests
import statsmodels.tsa.stattools as ts
import statsmodels.api as sm
import matplotlib.pyplot as plt
import datetime

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
def get_binance_symbols():
    url = 'https://fapi.binance.com/fapi/v1/ticker/price'
    r = requests.get(url)
    data = r.json()
    return pd.DataFrame(data)







def get_binance_asset(asset, c):
    url = 'http://fapi.binance.com/fapi/v1/klines?symbol=' + asset
    params = { 'limit': '700', 
               'interval': '1h'
               }
    r = requests.get(url, params)
    data = r.json()
    return pd.DataFrame(data)[c]




#assets = get_binance_symbols()['symbol']  #из-за новых монет тут лимит, возможно, придется уменьшить, потому как возникает ошибка или воспользоваться списком ниже

assets = ['BTCUSDT', 'XTZUSDT', 'LINKUSDT', 'BATUSDT', 'NEOUSDT', 'ZECUSDT', 'DASHUSDT', 'ETHUSDT', 'ETCUSDT', 'LTCUSDT', 'XLMUSDT', 'XRPUSDT', 'ONTUSDT', 'AKROUSDT', 'ZENUSDT', 'DYDXUSDT', 'CTSIUSDT', 'DUSKUSDT',

'BNBUSDT', 'XMRUSDT', 'BCHUSDT', 'EOSUSDT', 'THETAUSDT', 'WAVESUSDT', 'IOSTUSDT', 'ADAUSDT', 'DOTUSDT', 'QTUMUSDT', 'SXPUSDT', 'MKRUSDT', 'CRVUSDT', 'CHZUSDT', 'BTSUSDT', 'TLMUSDT', 'GALAUSDT', 'LPTUSDT',

'MATICUSDT', 'ENJUSDT', 'UNIUSDT', 'MKRUSDT', 'RUNEUSDT', 'BELUSDT', 'BALUSDT', 'REEFUSDT', '1INCHUSDT', 'COMPUSDT', 'SNXUSDT', 'TRBUSDT', 'SRMUSDT', 'LITUSDT', 'UNFIUSDT', 'AUDIOUSDT', 'ARUSDT', 'ENSUSDT',

'NEARUSDT', 'KAVAUSDT', 'STORJUSDT', 'TOMOUSDT', 'AAVEUSDT', 'ANKRUSDT', 'RENUSDT', 'KNCUSDT', 'RLCUSDT', 'ICXUSDT', 'BLZUSDT', 'FTMUSDT', 'HNTUSDT', 'DODOUSDT', 'RVNUSDT', 'RAYUSDT', 'KLAYUSDT', 'PEOPLEUSDT',

'SANDUSDT', 'ATOMUSDT', 'SOLUSDT', 'OGNUSDT', '1000SHIBUSDT', 'BAKEUSDT', 'SKLUSDT', 'FLMUSDT', 'RSRUSDT', 'LRCUSDT', 'CVCUSDT', 'CTKUSDT', 'AXSUSDT', 'ALPHAUSDT', 'SFPUSDT', 'C98USDT', 'ARPAUSDT', 'ANTUSDT',
          
'DOGEUSDT', 'AVAXUSDT', 'ALGOUSDT', 'ICPUSDT', 'FILUSDT', 'TRXUSDT', 'VETUSDT', 'EGLDUSDT', 'HBARUSDT', 'XEMUSDT', 'COTIUSDT', 'CHRUSDT', 'MANAUSDT', 'ALICEUSDT', 'ATAUSDT', 'NUUSDT', 'ROSEUSDT',
          
'IOTAUSDT', 'GRTUSDT', 'KSMUSDT', 'SUSHIUSDT', 'DENTUSDT', 'OCEANUSDT', 'STMXUSDT', 'CELRUSDT', 'HOTUSDT', 'KEEPUSDT', 'GTCUSDT', 'MTLUSDT', 'NKNUSDT', 'SCUSDT', 'DGBUSDT', 'ICPUSDT', 'LINAUSDT', 'ONEUSDT']


crypto_prices = pd.DataFrame()



for a in assets:
    a = a.strip()
    left = len(assets) - assets.index(a)
    print(f'Downloading {a}. {left} left')
    crypto_prices[a] = get_binance_asset(asset = a, c = 4).astype(float)
    crypto_prices['Time'] = get_binance_asset(asset = a, c = 0).astype('datetime64[ms]')
    
    

crypto_prices.set_index('Time', inplace=True)
#crypto_prices.index = pd.to_datetime(crypto_prices.index, unit='ms')

#print crypto_prices

crypto_prices.dropna(axis='columns', inplace=True)   # код от Алексея, удаляет колонки с пустыми значениями, фиксит предыдущий мой комментарий


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:50: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`


In [ ]:
data_analysis = {'Pairs': [],
                 'P-Value': []}

#data_ratios= {}

for stock1 in crypto_prices.columns:
    for stock2 in crypto_prices.columns:
        if stock1 != stock2:
            if (stock2, stock1) in data_analysis['Pairs']:
                continue

            pairs = stock1, stock2
            ratio = crypto_prices[stock1]/crypto_prices[stock2]
            pvalue = ts.adfuller(ratio)[1]
            #y = crypto_prices[stock1]
            #x = crypto_prices[stock2]
            #model = sm.OLS(y, sm.add_constant(x) ,missing='drop').fit()
            #pvalue = ts.adfuller(model.resid)[1]
            
            
            

            data_analysis['Pairs'].append(pairs)
            data_analysis['P-Value'].append(pvalue)

            #data_ratios[pairs] = ratio
            
data_analysis = pd.DataFrame(data_analysis).sort_values('P-Value')
data_analysis.set_index('Pairs')



,P-Value
Pairs,
"(NEARUSDT, MANAUSDT)",3.915480e-24
"(SRMUSDT, MANAUSDT)",5.459067e-19
"(ETHUSDT, MANAUSDT)",6.684753e-18
"(BCHUSDT, MANAUSDT)",8.528046e-17
"(FILUSDT, MANAUSDT)",2.982549e-15
...,...
"(BLZUSDT, COTIUSDT)",1.000000e+00
"(BLZUSDT, HBARUSDT)",1.000000e+00
"(BLZUSDT, FILUSDT)",1.000000e+00


In [ ]:
tradable_pairs_analysis = data_analysis[data_analysis['P-Value'] < 0.02].sort_values('P-Value')

tradable_pairs_analysis.set_index('Pairs')

,P-Value
Pairs,
"(NEARUSDT, MANAUSDT)",3.915480e-24
"(SRMUSDT, MANAUSDT)",5.459067e-19
"(ETHUSDT, MANAUSDT)",6.684753e-18
"(BCHUSDT, MANAUSDT)",8.528046e-17
"(FILUSDT, MANAUSDT)",2.982549e-15
...,...
"(TOMOUSDT, ALPHAUSDT)",1.974916e-02
"(DYDXUSDT, AXSUSDT)",1.992591e-02
"(EOSUSDT, ALPHAUSDT)",1.994274e-02




```
# Выбран кодовый формат
```



In [ ]:
# Get all tradable pairs into a list
stock_pairs = list(tradable_pairs_analysis['Pairs'].tolist())

# Show the Pairs
for pair1, pair2 in stock_pairs:
    print(f"BINANCE:{pair1}/BINANCE:{pair2},")

BINANCE:NEARUSDT/BINANCE:MANAUSDT,
BINANCE:SRMUSDT/BINANCE:MANAUSDT,
BINANCE:ETHUSDT/BINANCE:MANAUSDT,
BINANCE:BCHUSDT/BINANCE:MANAUSDT,
BINANCE:FILUSDT/BINANCE:MANAUSDT,
BINANCE:ETHUSDT/BINANCE:ENJUSDT,
BINANCE:RAYUSDT/BINANCE:MANAUSDT,
BINANCE:BALUSDT/BINANCE:MANAUSDT,
BINANCE:AXSUSDT/BINANCE:CELRUSDT,
BINANCE:ENJUSDT/BINANCE:MANAUSDT,
BINANCE:LPTUSDT/BINANCE:MANAUSDT,
BINANCE:MATICUSDT/BINANCE:MANAUSDT,
BINANCE:DOGEUSDT/BINANCE:MANAUSDT,
BINANCE:ALGOUSDT/BINANCE:MANAUSDT,
BINANCE:1000SHIBUSDT/BINANCE:MANAUSDT,
BINANCE:RUNEUSDT/BINANCE:MANAUSDT,
BINANCE:IOSTUSDT/BINANCE:ANKRUSDT,
BINANCE:EOSUSDT/BINANCE:ANKRUSDT,
BINANCE:BCHUSDT/BINANCE:DOTUSDT,
BINANCE:NEARUSDT/BINANCE:KAVAUSDT,
BINANCE:ETHUSDT/BINANCE:GALAUSDT,
BINANCE:CRVUSDT/BINANCE:MANAUSDT,
BINANCE:EOSUSDT/BINANCE:MANAUSDT,
BINANCE:UNIUSDT/BINANCE:KAVAUSDT,
BINANCE:BTCUSDT/BINANCE:KAVAUSDT,
BINANCE:TRXUSDT/BINANCE:SCUSDT,
BINANCE:XRPUSDT/BINANCE:MANAUSDT,
BINANCE:AVAXUSDT/BINANCE:MANAUSDT,
BINANCE:LTCUSDT/BINANCE:MANAUSDT,
BINA

In [ ]:
# Create a list of unique tradable stocks
list_stock1 = [stock[0] for stock in stock_pairs]
list_stock2 = [stock[1] for stock in stock_pairs]

for stock in list_stock2:
    list_stock1.append(stock)
    
unique_stock_list = list(set(list_stock1))

# Create a new DataFrame containing all market information for the tradable pairs
tradable_pairs_data = crypto_prices[unique_stock_list]
tradable_pairs_data.index = pd.to_datetime(tradable_pairs_data.index)


In [ ]:
data_ratios= {}
def get_ratios():
    for stock1 in tradable_pairs_data.columns:
        for stock2 in tradable_pairs_data.columns:
            if stock1 != stock2:
            
                pairs = stock1, stock2
                ratio = tradable_pairs_data[stock1].astype(float)/tradable_pairs_data[stock2].astype(float)
                data_ratios[pairs] = ratio
                
new_ratios = {}
def get_newratios():    
    for stock1 in new_prices.columns:
        for stock2 in new_prices.columns:
            if stock1 != stock2:
                pairs = stock1, stock2
                ratio = new_prices[stock1].astype(float)/new_prices[stock2].astype(float)
                new_ratios[pairs] = ratio
                

                
zscores = {}
def get_zscores():    
    for pair in data_ratios:
        zscore = (data_ratios[pair] - data_ratios[pair].mean()) / data_ratios[pair].std()
        zscores[pair] = zscore               






In [ ]:
#pd.set_option("display.max_rows", None, "display.max_columns", None)
#tradable_pairs_data
get_ratios()
get_zscores()
zscores

{('LPTUSDT', 'ETHUSDT'): Time
 2022-05-12 05:00:00   -0.315658
 2022-05-12 06:00:00   -1.139175
 2022-05-12 07:00:00   -1.090230
 2022-05-12 08:00:00   -0.631944
 2022-05-12 09:00:00   -0.394255
                          ...   
 2022-06-10 04:00:00   -0.706751
 2022-06-10 05:00:00   -0.717323
 2022-06-10 06:00:00   -0.825149
 2022-06-10 07:00:00   -0.738990
 2022-06-10 08:00:00   -0.978499
 Length: 700, dtype: float64, ('LPTUSDT', 'ARPAUSDT'): Time
 2022-05-12 05:00:00    1.303966
 2022-05-12 06:00:00    1.787907
 2022-05-12 07:00:00    1.484950
 2022-05-12 08:00:00    1.548123
 2022-05-12 09:00:00    1.701626
                          ...   
 2022-06-10 04:00:00   -1.454169
 2022-06-10 05:00:00   -1.525751
 2022-06-10 06:00:00   -1.572737
 2022-06-10 07:00:00   -1.519662
 2022-06-10 08:00:00   -1.615495
 Length: 700, dtype: float64, ('LPTUSDT', 'AXSUSDT'): Time
 2022-05-12 05:00:00   -0.318810
 2022-05-12 06:00:00   -0.671274
 2022-05-12 07:00:00   -1.542136
 2022-05-12 08:00:00   -1.

In [ ]:
from matplotlib import pyplot as plt

    
for pair in stock_pairs: 
    dev = 3
    plt.figure(figsize=(15,7))
    minwin = 1
    maxwin = 240
    ratios = data_ratios[pair]

    ratios_mavg5 = ratios.rolling(window=minwin, center=False).mean()
    ratios_mavg60 = ratios.rolling(window=maxwin, center=False).mean()
    std_60 = ratios.rolling(window=maxwin, center=False).std()
    zscore_60_5 = (ratios_mavg5 - ratios_mavg60)/std_60

    plt.plot(zscore_60_5.index, zscore_60_5)
    plt.title('Ratio {}'.format(pair))
    plt.axhline(0)
    plt.axhline(dev, linestyle='--')
    plt.axhline(-dev, linestyle='--')
    plt.legend(['Rolling Ratio z-Score', 'Mean', dev, -dev])
    plt.show()
        
